In [1]:
import os
import zipfile
import time
import pickle
import gc

import pandas as pd
import numpy as np
from tqdm import tqdm

from utils import load_pickle, dump_pickle, get_feature_value, feature_spearmanr, feature_target_spearmanr, addCrossFeature, calibration
from utils import raw_data_path, feature_data_path, cache_pkl_path, analyse

In [2]:
features=[
#          'item_id',
         'item_brand_id',
         'item_city_id',
         'item_price_level',
         'item_sales_level',
         'item_collected_level',
         'item_pv_level',
#          'user_id',
         'user_gender_id',
         'user_age_level',
         'user_occupation_id',
         'user_star_level',
#          'context_id',
         'context_timestamp',
         'context_page_id',
         'shop_id',
         'shop_review_num_level',
         'shop_review_positive_rate',
         'shop_star_level',
         'shop_score_service',
         'shop_score_delivery',
         'shop_score_description',
#          'day',
#          'hour',
#          'minute',
         'category2_label',
         'category3_label',
#2-9
         'user_gender_id_item_price_level',
#          'user_gender_id_item_sales_level',
#          'user_gender_id_shop_star_level',
#          'user_gender_id_shop_review_num_level',
#          'user_gender_id_shop_review_positive_rate',
         'user_gender_id_category2_label',
         'user_gender_id_category3_label',
#          'user_gender_id_context_page_id',
#          'user_gender_id_hour',
         'user_age_level_item_price_level',
#          'user_age_level_item_sales_level',
#          'user_age_level_shop_star_level',
#          'user_age_level_shop_review_num_level',
#          'user_age_level_shop_review_positive_rate',
         'user_age_level_category2_label',
         'user_age_level_category3_label',
#          'user_age_level_context_page_id',
#          'user_age_level_hour',
         'user_occupation_id_item_price_level',
#          'user_occupation_id_item_sales_level',
#          'user_occupation_id_shop_star_level',
#          'user_occupation_id_shop_review_num_level',
#          'user_occupation_id_shop_review_positive_rate',
         'user_occupation_id_category2_label',
         'user_occupation_id_category3_label',
#          'user_occupation_id_context_page_id',
#          'user_occupation_id_hour',
         'user_star_level_item_price_level',
#          'user_star_level_item_sales_level',
#          'user_star_level_shop_star_level',
#          'user_star_level_shop_review_num_level',
#          'user_star_level_shop_review_positive_rate',
         'user_star_level_category2_label',
         'user_star_level_category3_label',
#          'user_star_level_context_page_id',
#          'user_star_level_hour',
         'user_gender_id_user_age_level',
         'user_gender_id_user_occupation_id',
         'user_gender_id_user_star_level',
         'user_age_level_user_occupation_id',
         'user_age_level_user_star_level',
         'user_occupation_id_user_star_level',
    
    
# category
#          'item_price_level_bin',
#          'item_sales_level_bin',
#          'item_collected_level_bin',
#          'item_pv_level_bin',
#          'shop_review_num_level_bin',
#          'shop_review_positive_rate_bin',
#          'shop_star_level_bin',
#          'shop_score_service_bin',
#          'shop_score_delivery_bin',
#          'shop_score_description_bin',
#          'hour_bin',

         'item_property_topic_k_15',
    
#2-1
#         'user_category2_label_click_day',
#         'user_category3_label_click_day',
#         'user_shop_id_click_day',
#         'user_item_id_click_day',
#         'user_item_brand_id_click_day',
#         'user_context_page_id_click_day',
#         'user_item_price_level_bin_click_day',
#         'user_item_sales_level_bin_click_day',
#         'user_item_property_topic_k_15_click_day',
    
#         'user_category2_label_click_hour',
#         'user_category3_label_click_hour',
#         'user_shop_id_click_hour',
#         'user_item_id_click_hour',
#         'user_item_brand_id_click_hour',
#         'user_context_page_id_click_hour',
#         'user_item_price_level_bin_click_hour',
#         'user_item_sales_level_bin_click_hour',
#         'user_item_property_topic_k_15_click_hour',
    
    
#2-2    
#          'user_id_click_day_mean',
#          'category2_label_click_day_mean',
#          'category3_label_click_day_mean',
#          'shop_id_click_day_mean',
#          'item_id_click_day_mean',
#          'item_brand_id_click_day_mean',
#          'context_page_id_click_day_mean',
    
    
#2-3
        
#         'user_click_rank_day',
#         'user_click_true_rank_day',
#         'user_click_interval_first_day',
#         'user_click_interval_last_day',
        'user_click_time_gap_before',
        'user_click_time_gap_after',
    
#         'user_click_rank_global',
#         'user_click_true_rank_global',
#         'user_click_interval_first_global',
#         'user_click_interval_last_global',
#         'user_click_time_gap_before_global',
#         'user_click_time_gap_after_global',
    
#         'user_category2_label_click_rank_day',
#         'user_category2_label_click_true_rank_day',
#         'user_category2_label_first_click_interval_day',
#         'user_category2_label_last_click_interval_day',
        'user_category2_label_click_time_gap_before_day',
        'user_category2_label_click_time_gap_after_day',
#         'user_category3_label_click_rank_day',
#         'user_category3_label_click_true_rank_day',
#         'user_category3_label_first_click_interval_day',
#         'user_category3_label_last_click_interval_day',
        'user_category3_label_click_time_gap_before_day',
        'user_category3_label_click_time_gap_after_day',
#         'user_shop_id_click_rank_day',
#         'user_shop_id_click_true_rank_day',
#         'user_shop_id_first_click_interval_day',
#         'user_shop_id_last_click_interval_day',
        'user_shop_id_click_time_gap_before_day',
        'user_shop_id_click_time_gap_after_day',
#         'user_item_id_click_rank_day',
#         'user_item_id_click_true_rank_day',
#         'user_item_id_first_click_interval_day',
#         'user_item_id_last_click_interval_day',
        'user_item_id_click_time_gap_before_day',
        'user_item_id_click_time_gap_after_day',
#         'user_item_brand_id_click_rank_day',
#         'user_item_brand_id_click_true_rank_day',
#         'user_item_brand_id_first_click_interval_day',
#         'user_item_brand_id_last_click_interval_day',
        'user_item_brand_id_click_time_gap_before_day',
        'user_item_brand_id_click_time_gap_after_day',
#         'user_item_property_topic_k_15_click_rank_day',
#         'user_item_property_topic_k_15_click_true_rank_day',
#         'user_item_property_topic_k_15_first_click_interval_day',
#         'user_item_property_topic_k_15_last_click_interval_day',
        'user_item_property_topic_k_15_click_time_gap_before_day',
        'user_item_property_topic_k_15_click_time_gap_after_day',
    
#         'user_category2_label_click_rank_global',
#         'user_category2_label_click_true_rank_global',
#         'user_category2_label_first_click_interval_global',
#         'user_category2_label_last_click_interval_global',
#         'user_category2_label_click_time_gap_before_global',
#         'user_category2_label_click_time_gap_after_global',
#         'user_category3_label_click_rank_global',
#         'user_category3_label_click_true_rank_global',
#         'user_category3_label_first_click_interval_global',
#         'user_category3_label_last_click_interval_global',
#         'user_category3_label_click_time_gap_before_global',
#         'user_category3_label_click_time_gap_after_global',
#         'user_shop_id_click_rank_global',
#         'user_shop_id_click_true_rank_global',
#         'user_shop_id_first_click_interval_global',
#         'user_shop_id_last_click_interval_global',
#         'user_shop_id_click_time_gap_before_global',
#         'user_shop_id_click_time_gap_after_global',
#         'user_item_id_click_rank_global',
#         'user_item_id_click_true_rank_global',
#         'user_item_id_first_click_interval_global',
#         'user_item_id_last_click_interval_global',
#         'user_item_id_click_time_gap_before_global',
#         'user_item_id_click_time_gap_after_global',
#         'user_item_brand_id_click_rank_global',
#         'user_item_brand_id_click_true_rank_global',
#         'user_item_brand_id_first_click_interval_global',
#         'user_item_brand_id_last_click_interval_global',
#         'user_item_brand_id_click_time_gap_before_global',
#         'user_item_brand_id_click_time_gap_after_global',
#         'user_item_property_topic_k_15_click_rank_global',
#         'user_item_property_topic_k_15_click_true_rank_global',
#         'user_item_property_topic_k_15_first_click_interval_global',
#         'user_item_property_topic_k_15_last_click_interval_global',
#         'user_item_property_topic_k_15_click_time_gap_before_global',
#         'user_item_property_topic_k_15_click_time_gap_after_global',
    
#2-4
        'item_property_topic_0',
        'item_property_topic_1',
        'item_property_topic_2',
        'item_property_topic_3',
        'item_property_topic_4',
        'item_property_topic_5',
        'item_property_topic_6',
        'item_property_topic_7',
        'item_property_topic_8',
        'item_property_topic_9',
        'item_property_topic_10',
        'item_property_topic_11',
        'item_property_topic_12',
        'item_property_topic_13',
        'item_property_topic_14',
        'property_sim',
        'category_predict_rank',

    
    
#2-5
#     交叉特征历史ctr

        'user_gender_id_item_price_level_smooth_CTR',
#         'user_gender_id_item_sales_level_smooth_CTR',
#         'user_gender_id_shop_star_level_smooth_CTR',
#         'user_gender_id_shop_review_num_level_smooth_CTR',
#         'user_gender_id_shop_review_positive_rate_smooth_CTR',
        'user_gender_id_category2_label_smooth_CTR',
        'user_gender_id_category3_label_smooth_CTR',
        'user_age_level_item_price_level_smooth_CTR',
#         'user_age_level_item_sales_level_smooth_CTR',
#         'user_age_level_shop_star_level_smooth_CTR',
#         'user_age_level_shop_review_num_level_smooth_CTR',
#         'user_age_level_shop_review_positive_rate_smooth_CTR',
        'user_age_level_category2_label_smooth_CTR',
        'user_age_level_category3_label_smooth_CTR',
        'user_occupation_id_item_price_level_smooth_CTR',
#         'user_occupation_id_item_sales_level_smooth_CTR',
#         'user_occupation_id_shop_star_level_smooth_CTR',
#         'user_occupation_id_shop_review_num_level_smooth_CTR',
#         'user_occupation_id_shop_review_positive_rate_smooth_CTR',
        'user_occupation_id_category2_label_smooth_CTR',
        'user_occupation_id_category3_label_smooth_CTR',
#         'user_star_level_item_price_level_smooth_CTR',
#         'user_star_level_item_sales_level_smooth_CTR',
#         'user_star_level_shop_star_level_smooth_CTR',
#         'user_star_level_shop_review_num_level_smooth_CTR',
#         'user_star_level_shop_review_positive_rate_smooth_CTR',
#         'user_star_level_category2_label_smooth_CTR',
#         'user_star_level_category3_label_smooth_CTR',
        'user_gender_id_user_age_level_smooth_CTR',
        'user_gender_id_user_occupation_id_smooth_CTR',
        'user_gender_id_user_star_level_smooth_CTR',
        'user_age_level_user_occupation_id_smooth_CTR',
        'user_age_level_user_star_level_smooth_CTR',
        'user_occupation_id_user_star_level_smooth_CTR',
    
    
#     交叉特征当前小时历史ctr
#         'user_gender_id_item_price_level_hour_CTR',
# #         'user_gender_id_item_sales_level_hour_CTR',
# #         'user_gender_id_shop_star_level_hour_CTR',
# #         'user_gender_id_shop_review_num_level_hour_CTR',
# #         'user_gender_id_shop_review_positive_rate_hour_CTR',
#         'user_gender_id_category2_label_hour_CTR',
#         'user_gender_id_category3_label_hour_CTR',
#         'user_age_level_item_price_level_hour_CTR',
# #         'user_age_level_item_sales_level_hour_CTR',
# #         'user_age_level_shop_star_level_hour_CTR',
# #         'user_age_level_shop_review_num_level_hour_CTR',
# #         'user_age_level_shop_review_positive_rate_hour_CTR',
#         'user_age_level_category2_label_hour_CTR',
#         'user_age_level_category3_label_hour_CTR',
#         'user_occupation_id_item_price_level_hour_CTR',
# #         'user_occupation_id_item_sales_level_hour_CTR',
# #         'user_occupation_id_shop_star_level_hour_CTR',
# #         'user_occupation_id_shop_review_num_level_hour_CTR',
# #         'user_occupation_id_shop_review_positive_rate_hour_CTR',
#         'user_occupation_id_category2_label_hour_CTR',
#         'user_occupation_id_category3_label_hour_CTR',
# #         'user_star_level_item_price_level_hour_CTR',
# #         'user_star_level_item_sales_level_hour_CTR',
# #         'user_star_level_shop_star_level_hour_CTR',
# #         'user_star_level_shop_review_num_level_hour_CTR',
# #         'user_star_level_shop_review_positive_rate_hour_CTR',
# #         'user_star_level_category2_label_hour_CTR',
# #         'user_star_level_category3_label_hour_CTR',
#         'user_gender_id_user_age_level_hour_CTR',
#         'user_gender_id_user_occupation_id_hour_CTR',
#         'user_gender_id_user_star_level_hour_CTR',
#         'user_age_level_user_occupation_id_hour_CTR',
#         'user_age_level_user_star_level_hour_CTR',
#         'user_occupation_id_user_star_level_hour_CTR',
    
    
#     单特征历史点击与ctr    
         'user_id_smooth_I',
         'user_id_smooth_C',
         'user_id_smooth_CTR',
         'item_id_smooth_I',
         'item_id_smooth_C',
         'item_id_smooth_CTR',
#          'item_brand_id_smooth_I',
#          'item_brand_id_smooth_C',
         'item_brand_id_smooth_CTR',
#          'category2_label_smooth_I',
#          'category2_label_smooth_C',
         'category2_label_smooth_CTR',
#          'category3_label_smooth_I',
#          'category3_label_smooth_C',
         'category3_label_smooth_CTR',
#          'context_page_id_smooth_I',
#          'context_page_id_smooth_C',
         'context_page_id_smooth_CTR',
#          'shop_id_smooth_I',
#          'shop_id_smooth_C',
         'shop_id_smooth_CTR',
#          'item_sales_level_bin_smooth_I',
#          'item_sales_level_bin_smooth_C',
         'item_sales_level_bin_smooth_CTR',
#          'item_price_level_bin_smooth_I',
#          'item_price_level_bin_smooth_C',
         'item_price_level_bin_smooth_CTR',
#          'item_collected_level_bin_smooth_I',
#          'item_collected_level_bin_smooth_C',
         'item_collected_level_bin_smooth_CTR',
#          'item_pv_level_bin_smooth_I',
#          'item_pv_level_bin_smooth_C',
         'item_pv_level_bin_smooth_CTR',
#          'shop_review_num_level_bin_smooth_I',
#          'shop_review_num_level_bin_smooth_C',
         'shop_review_num_level_bin_smooth_CTR',
#          'shop_review_positive_rate_bin_smooth_I',
#          'shop_review_positive_rate_bin_smooth_C',
         'shop_review_positive_rate_bin_smooth_CTR',
#          'shop_star_level_bin_smooth_I',
#          'shop_star_level_bin_smooth_C',
         'shop_star_level_bin_smooth_CTR',
#          'shop_score_service_bin_smooth_I',
#          'shop_score_service_bin_smooth_C',
         'shop_score_service_bin_smooth_CTR',
#          'shop_score_delivery_bin_smooth_I',
#          'shop_score_delivery_bin_smooth_C',
         'shop_score_delivery_bin_smooth_CTR',
#          'shop_score_description_bin_smooth_I',
#          'shop_score_description_bin_smooth_C',
         'shop_score_description_bin_smooth_CTR',
#          'hour_smooth_I',
#          'hour_smooth_C',
#          'hour_smooth_CTR',
    
    
#          'user_id_hour_CTR',
#          'item_id_hour_CTR',
#          'item_brand_id_hour_CTR',
#          'category2_label_hour_CTR',
#          'category3_label_hour_CTR',
#          'context_page_id_hour_CTR',
#          'shop_id_hour_CTR',
#          'item_sales_level_bin_hour_CTR',
#          'item_price_level_bin_hour_CTR',
#          'item_collected_level_bin_hour_CTR',
#          'item_pv_level_bin_hour_CTR',
#          'shop_review_num_level_bin_hour_CTR',
#          'shop_review_positive_rate_bin_hour_CTR',
#          'shop_star_level_bin_hour_CTR',
#          'shop_score_service_bin_hour_CTR',
#          'shop_score_delivery_bin_hour_CTR',
#          'shop_score_description_bin_hour_CTR',
    
    
#     用户交叉特征历史ctr
        'user_id_item_id_history_I',
        'user_id_item_id_history_C',
#         'user_id_item_id_history_CTR',
        'user_id_item_brand_id_history_I',
        'user_id_item_brand_id_history_C',
#         'user_id_item_brand_id_history_CTR',
        'user_id_category2_label_history_I',
        'user_id_category2_label_history_C',
#         'user_id_category2_label_history_CTR',
        'user_id_category3_label_history_I',
        'user_id_category3_label_history_C',
#         'user_id_category3_label_history_CTR',
        'user_id_shop_id_history_I',
        'user_id_shop_id_history_C',
#         'user_id_shop_id_history_CTR',
#         'user_id_item_sales_level_bin_history_I',
#         'user_id_item_sales_level_bin_history_C',
#         'user_id_item_sales_level_bin_history_CTR',
#         'user_id_item_price_level_bin_history_I',
#         'user_id_item_price_level_bin_history_C',
#         'user_id_item_price_level_bin_history_CTR',

    
#     各种id属性与用户属性交叉历史ctr
        'user_gender_id_shop_id_smooth_CTR',
        'user_gender_id_item_id_smooth_CTR',
        'user_gender_id_item_brand_id_smooth_CTR',
        'user_age_level_shop_id_smooth_CTR',
        'user_age_level_item_id_smooth_CTR',
        'user_age_level_item_brand_id_smooth_CTR',
        'user_occupation_id_shop_id_smooth_CTR',
        'user_occupation_id_item_id_smooth_CTR',
        'user_occupation_id_item_brand_id_smooth_CTR',
#         'user_star_level_shop_id_smooth_CTR',
#         'user_star_level_item_id_smooth_CTR',
#         'user_star_level_item_brand_id_smooth_CTR',
    
    
    
#2-6
         'shop_id_user_gender_id_click_rate',
         'shop_id_user_age_level_click_rate',
         'shop_id_user_occupation_id_click_rate',
#          'shop_id_user_star_level_click_rate',
#          'item_id_user_gender_id_click_rate',
#          'item_id_user_age_level_click_rate',
#          'item_id_user_occupation_id_click_rate',
#          'item_id_user_star_level_click_rate',
         'item_brand_id_user_gender_id_click_rate',
         'item_brand_id_user_age_level_click_rate',
         'item_brand_id_user_occupation_id_click_rate',
#          'item_brand_id_user_star_level_click_rate',
         'category2_label_user_gender_id_click_rate',
         'category2_label_user_age_level_click_rate',
         'category2_label_user_occupation_id_click_rate',
#          'category2_label_user_star_level_click_rate',
         'category3_label_user_gender_id_click_rate',
         'category3_label_user_age_level_click_rate',
         'category3_label_user_occupation_id_click_rate',
#          'category3_label_user_star_level_click_rate',
#          'hour_user_gender_id_click_rate',
#          'hour_user_age_level_click_rate',
#          'hour_user_occupation_id_click_rate',
#          'hour_user_star_level_click_rate',
#          'item_sales_level_bin_user_gender_id_click_rate',
#          'item_sales_level_bin_user_age_level_click_rate',
#          'item_sales_level_bin_user_occupation_id_click_rate',
#          'item_sales_level_bin_user_star_level_click_rate',
         'item_price_level_bin_user_gender_id_click_rate',
         'item_price_level_bin_user_age_level_click_rate',
         'item_price_level_bin_user_occupation_id_click_rate',
#          'item_price_level_bin_user_star_level_click_rate',
    
    
#2-7
#          'user_id_click_day',
#          'item_id_click_day',
#          'item_brand_id_click_day',
#          'category2_label_click_day',
#          'category3_label_click_day',
#          'context_page_id_click_day',
#          'shop_id_click_day',
#          'item_property_topic_k_15_click_day',
    
#          'user_id_click_hour',
#          'item_id_click_hour',
#          'item_brand_id_click_hour',
#          'category2_label_click_hour',
#          'category3_label_click_hour',
#          'context_page_id_click_hour',
#          'shop_id_click_hour',
#          'item_property_topic_k_15_click_hour',
    
    
#2-8
#          'user_item_id_pre_click',
         'user_item_id_continue_click',
#          'user_item_brand_id_pre_click',
         'user_item_brand_id_continue_click',
#          'user_shop_id_pre_click',
         'user_shop_id_continue_click',
#          'user_category2_label_pre_click',
         'user_category2_label_continue_click',
#          'user_category3_label_pre_click',
         'user_category3_label_continue_click',
#          'user_item_property_topic_k_15_pre_click',
         'user_item_property_topic_k_15_continue_click',
#          'user_item_sales_level_bin_pre_click',
#          'user_item_sales_level_bin_continue_click',
#          'user_item_price_level_bin_pre_click',
#          'user_item_price_level_bin_continue_click',
    
    
#2-10  
         'user_category2_label_future_2min',
#          'user_category2_label_future_15min',
         'user_category3_label_future_2min',
#          'user_category3_label_future_15min',
         'user_shop_id_future_2min',
#          'user_shop_id_future_15min',
         'user_item_id_future_2min',
#          'user_item_id_future_15min',
         'user_item_brand_id_future_2min',
#          'user_item_brand_id_future_15min',
         'user_item_property_topic_k_15_future_2min',
#          'user_item_property_topic_k_15_future_15min',
#          'user_item_sales_level_bin_future_2min',
#          'user_item_sales_level_bin_future_15min',
#          'user_item_price_level_bin_future_2min',
#          'user_item_price_level_bin_future_15min'
    
#          'shop_id_user_gender_id_click_rate_hour',
#          'shop_id_user_age_level_click_rate_hour',
#          'shop_id_user_occupation_id_click_rate_hour',
#          'shop_id_user_star_level_click_rate_hour',
#          'item_id_user_gender_id_click_rate_hour',
#          'item_id_user_age_level_click_rate_hour',
#          'item_id_user_occupation_id_click_rate_hour',
#          'item_id_user_star_level_click_rate_hour',
#          'item_brand_id_user_gender_id_click_rate_hour',
#          'item_brand_id_user_age_level_click_rate_hour',
#          'item_brand_id_user_occupation_id_click_rate_hour',
#          'item_brand_id_user_star_level_click_rate_hour',
#          'category2_label_user_gender_id_click_rate_hour',
#          'category2_label_user_age_level_click_rate_hour',
#          'category2_label_user_occupation_id_click_rate_hour',
#          'category2_label_user_star_level_click_rate_hour',
#          'category3_label_user_gender_id_click_rate_hour',
#          'category3_label_user_age_level_click_rate_hour',
#          'category3_label_user_occupation_id_click_rate_hour',
#          'category3_label_user_star_level_click_rate_hour',
#          'item_sales_level_bin_user_gender_id_click_rate_hour',
#          'item_sales_level_bin_user_age_level_click_rate_hour',
#          'item_sales_level_bin_user_occupation_id_click_rate_hour',
#          'item_sales_level_bin_user_star_level_click_rate_hour',
#          'item_price_level_bin_user_gender_id_click_rate_hour',
#          'item_price_level_bin_user_age_level_click_rate_hour',
#          'item_price_level_bin_user_occupation_id_click_rate_hour',
#          'item_price_level_bin_user_star_level_click_rate_hour',
    
    
         'user_category2_label_before_2min',
#          'user_category2_label_before_15min',
         'user_category2_label_before_1hour',
         'user_category3_label_before_2min',
#          'user_category3_label_before_15min',
         'user_category3_label_before_1hour',
         'user_shop_id_before_2min',
#          'user_shop_id_before_15min',
         'user_shop_id_before_1hour',
         'user_item_id_before_2min',
#          'user_item_id_before_15min',
         'user_item_id_before_1hour',
         'user_item_brand_id_before_2min',
#          'user_item_brand_id_before_15min',
         'user_item_brand_id_before_1hour',
#          'user_item_sales_level_bin_before_2min',
#          'user_item_sales_level_bin_before_15min',
#          'user_item_sales_level_bin_before_1hour',
#          'user_item_price_level_bin_before_2min',
#          'user_item_price_level_bin_before_15min',
#          'user_item_price_level_bin_before_1hour',
         'user_item_property_topic_k_15_before_2min',
#          'user_item_property_topic_k_15_before_15min',
         'user_item_property_topic_k_15_before_1hour'
         ]
len(features)

165

In [3]:
all_data_path = feature_data_path + 'all_data_all_features_new.pkl'
all_data = load_pickle(all_data_path)

target = 'is_trade'

In [8]:
categorical_feature = [
             'item_price_level',
             'item_sales_level',
             'item_collected_level',
             'item_pv_level',
           
            'user_age_level',
            'user_gender_id',
#             'user_occupation_id',
            'user_star_level',
            'item_property_topic_k_15',
    
            'shop_review_num_level',
            'shop_star_level',
    
#             'category2_label',
#             'category3_label',

    
#             'user_click_rank_day',
#             'user_category2_label_click_rank_day',
#             'user_category3_label_click_rank_day',
#             'user_shop_id_click_rank_day',
#             'user_item_id_click_rank_day',
#             'user_item_brand_id_click_rank_day',
    
# #          'user_item_id_pre_click',
#          'user_item_id_continue_click',
# #          'user_item_brand_id_pre_click',
#          'user_item_brand_id_continue_click',
# #          'user_shop_id_pre_click',
#          'user_shop_id_continue_click',
# #          'user_category2_label_pre_click',
#          'user_category2_label_continue_click',
# #          'user_category3_label_pre_click',
#          'user_category3_label_continue_click',
# #          'user_item_property_topic_k_15_pre_click',
#          'user_item_property_topic_k_15_continue_click',
    
            'category_predict_rank',
    

]
len(categorical_feature)

11

In [5]:
data = all_data[(all_data.day == 7)&(all_data.hour>-1)]
# data = all_data[(all_data.day == 7)]

In [6]:
train_data = data[(data.is_trade != -1) & (data.is_trade != -2)]
test_data = data[data.is_trade == -2]
# del all_data
# # del data
# gc.collect()
print(train_data.shape)
print(test_data.shape)

(1077175, 514)
(1209768, 514)


In [ ]:
from sklearn.metrics import log_loss
import lightgbm as lgb

lgb_train_data = lgb.Dataset(
    train_data[features], label=train_data[target], feature_name=features,categorical_feature=categorical_feature)


param = {'application': 'binary',
         'metric': 'binary_logloss',
         
         'num_iterations':1000,
         'learning_rate': 0.05,
         
         'max_depth': 4,
         'num_leaves': 16,

#          'min_data_in_leaf': 200,
#          'min_sum_hessian_in_leaf': 0.1,
#          'min_gain_to_split': 0.1,

         'feature_fraction': 0.8,
         'bagging_fraction': 0.8,
         'bagging_freq': 1,

         'lambda_l2': 15,
         'max_bin': 63,

         'device': 'gpu',
         'gpu_use_dp': False,
         }


valid_sets = [lgb_train_data,]

bst = lgb.train(param, lgb_train_data, valid_sets=valid_sets, verbose_eval=50,categorical_feature=categorical_feature)


test_data['predicted_score'] = bst.predict(test_data[features])

test_data[['instance_id', 'predicted_score']].to_csv(
    '201805013_test.txt', index=False, sep=' ')


/home/qwc/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:99: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[50]	training's binary_logloss: 0.188262
[100]	training's binary_logloss: 0.173262
[150]	training's binary_logloss: 0.17145
[200]	training's binary_logloss: 0.170478
[250]	training's binary_logloss: 0.16981
[300]	training's binary_logloss: 0.169269
[350]	training's binary_logloss: 0.168817
[400]	training's binary_logloss: 0.168388
[450]	training's binary_logloss: 0.16804
[500]	training's binary_logloss: 0.167692
[550]	training's binary_logloss: 0.16737
[600]	training's binary_logloss: 0.167078
[650]	training's binary_logloss: 0.166781
[700]	training's binary_logloss: 0.166501
[750]	training's binary_logloss: 0.166243
[800]	training's binary_logloss: 0.165969
[850]	training's binary_logloss: 0.165721
[900]	training's binary_logloss: 0.165481
[950]	training's binary_logloss: 0.16522
[1000]	training's binary_logloss: 0.164973


In [11]:
a = pd.read_csv('201805013_test.txt', sep=' ')
b = a.predicted_score
c = float(sum(b)/float(len(b)))
c

0.04451811126863975

In [10]:
test_data['predicted_score'] = bst.predict(test_data[features])
ctr = test_data.groupby(['hour'])['predicted_score'].mean().reset_index().rename(columns={0: 'ctr'})
ctr

/home/qwc/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,hour,predicted_score
0,12,0.042488
1,13,0.042679
2,14,0.042380
3,15,0.042671
4,16,0.043277
5,17,0.044023
6,18,0.043958
7,19,0.044230
8,20,0.044938
9,21,0.045812


In [7]:
test_data[-10:]

,index,instance_id,item_id,item_brand_id,item_city_id,item_price_level,item_sales_level,item_collected_level,item_pv_level,user_id,...,user_item_id_continue_click,user_item_brand_id_pre_click,user_item_brand_id_continue_click,user_shop_id_pre_click,user_shop_id_continue_click,user_category2_label_pre_click,user_category2_label_continue_click,user_category3_label_pre_click,user_category3_label_continue_click,predicted_score
3531496,10951914,9220740874147378850,6025024564727510475,7838285046767229711,1019055478500227370,6,13,14,16,858518175562480456,...,1,0,1,0,1,0,1,1,4,0.075031
3531497,10951915,9220743669874614059,2708682881807207463,-1,3819392654129628501,5,7,6,14,2962330780682476331,...,1,0,1,0,1,1,2,1,5,0.057073
3531498,10951916,9221442632889108512,1076358075351846977,7964598119390198442,4918413420989329604,6,12,11,18,2652425316764384351,...,1,0,1,0,1,0,1,0,1,0.080356
3531499,10951917,9221649774284923257,981677653924611538,3755890348736743152,5161571070109172606,7,15,15,20,8268324770556436553,...,1,0,1,0,1,0,1,0,1,0.187027
3531500,10951918,9222014295467123530,1728983024300190044,7838285046767229711,196257267849351217,8,10,11,17,4626840049806596392,...,1,0,1,0,1,0,1,0,1,0.026803
3531501,10951919,9222465482510638693,3777455802965897080,5664460045239319053,8034210119001863030,7,10,12,15,8706200347186243583,...,1,0,1,0,1,0,1,0,1,0.022901
3531502,10951920,9222508164132941135,8403872358944866054,3766406943951954790,5918626470536001929,4,12,12,17,354376871364486592,...,1,0,1,0,1,0,1,0,1,0.063787
3531503,10951921,9222510657169993491,3404305105333934063,4181236264170900506,548352491538518780,7,9,11,18,3795475028342670753,...,1,0,1,0,1,0,1,0,1,0.017021
3531504,10951922,9222551255908347950,3730067091891558780,7838285046767229711,3819392654129628501,4,12,11,17,4402860134210132394,...,1,0,1,0,1,1,2,1,2,0.065972
3531505,10951923,9223162707596616022,3785008483302063270,1949840372704254450,5170960014354231267,7,12,11,17,6872328330615894369,...,1,0,1,0,1,0,1,0,1,0.041179


In [7]:
predict_depth_5 = pd.read_csv('20180503_qwc_467_2100.txt', sep=' ')
predict_depth_4 = pd.read_csv('20180502_qwc_1900.txt', sep=' ')

predict_depth_5['predicted_score'] = predict_depth_5['predicted_score'] * 0.5 + predict_depth_4['predicted_score'] * 0.5


predict_depth_5[['instance_id', 'predicted_score']].to_csv(
    '20180503-combination.txt', index=False, sep=' ')